## PaliGemma Fine-tuning

Pretrained Paligemma를 파인튜닝하여 딥페이크 기술로 생성된 이미지를 분류하는 모델을 생성


### 환경 설정

In [ ]:
!pip install torch
!pip install transformers
!pip install peft
!pip install trl
!pip install -U bitsandbytes
!pip install datasets
!pip install accelerate

Huggingface 로그인  
- paligemma에 대한 read 권한 확보
- 로그인 한 계정에 대하여 파인튜닝 모델 업로드 권한 확보

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### 학습 준비

학습 데이터셋 로드

In [ ]:
from datasets import load_dataset

ds = load_dataset("JamieWithofs/Deepfake-and-real-images")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/624 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/426M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/436M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/424M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/116M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/391M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/140002 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10905 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/39428 [00:00<?, ? examples/s]

In [ ]:
ds['train']

Dataset({
    features: ['image', 'label'],
    num_rows: 140002
})

In [ ]:
train_ds = ds['test'] # 학습 환경을 고려하여 적은 데이터셋을 학습 데이터로 선정

In [ ]:
train_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 10905
})

In [ ]:
question_make = ['Is this image made by AI? Yes(0) or No(1)' for i in range(len(train_ds['label']))] # paligemma 학습을 위하여 question 컬럼 생성
train_ds = train_ds.add_column("question", question_make) # train_ds에 question 컬럼 추가

In [ ]:
train_ds

Dataset({
    features: ['image', 'label', 'question'],
    num_rows: 10905
})

PaliGemmaProcessor는 PaliGemma 모델과 함께 사용하는 프로세서로, 모델의 입력 데이터를 적절히 전처리하고 모델의 출력을 후처리하는 역할을 수행  




In [ ]:
from transformers import PaliGemmaProcessor
model_id = "google/paligemma-3b-pt-224"
processor = PaliGemmaProcessor.from_pretrained(model_id)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# BLIP 모델과 프로세서 불러오기
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

# 이미지 설명 생성 함수
def generate_image_description(image):
    # 이미지를 텍스트 형식으로 전처리
    inputs = blip_processor(images=image, return_tensors="pt")

    # 설명 생성
    out = blip_model.generate(**inputs)

    # 설명을 텍스트로 변환
    description = blip_processor.decode(out[0], skip_special_tokens=True)
    return description

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
train_ds[0]['label']

0

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# 예시 이미지 로드
# plt.imshow(train_ds[0]['image'].convert("RGB"))  # 사용자가 제공하는 이미지

# 이미지 설명 생성
description = generate_image_description(train_ds[0]['image'].convert("RGB"))
print("Image Description:", description)

Image Description: an older woman in a blanket


In [ ]:
import torch
device = "cuda"

image_token = processor.tokenizer.convert_tokens_to_ids("<image>")
'''
collate_fn 함수는 여러 데이터 샘플을 배치로 묶기 위한 역할.
이 함수는 데이터 로더에서 호출되어 입력 데이터를 전처리합니다.
'''
def collate_fn(examples):
    texts = [example["question"] for example in examples]
    labels= [str(example['label']) for example in examples]
    images = [example["image"].convert("RGB") for example in examples]

     # 이미지 설명 생성
    image_descriptions = [generate_image_description(image) for image in images]

    # 텍스트와 이미지 설명 결합
    texts_with_descriptions = ["Description: There's " + description + "In this image. \nQuestion: " +text for text, description in zip(texts, image_descriptions)]

    tokens = processor(text=texts, images=images, suffix=labels,
                    return_tensors="pt", padding="longest",
                    tokenize_newline_separately=False)

    tokens = tokens.to(torch.bfloat16).to(device)
    return tokens


참고한 깃허브의 코드를 살펴보면 Paligemma를 학습하는데 사용된 데이터셋과 해당 코드에서 사용하는 VQA 데이터셋이 유사하기에,  
image encoder 부분을 파인튜닝 하지 않고 텍스트 디코더 부분만을 파인튜닝하도록 설정하였다고 되어있다.  
이에 아래와 같이 특정 파라미터의 업데이트를 False로 설정하였다.  

※ 금번 딥페이크 생성 이미지 분류 역시 데이터셋이 일반적인 얼굴 사진으로 구성된 데이터셋이므로, 동일한 구성으로 설정

In [ ]:
from transformers import PaliGemmaForConditionalGeneration
import torch

model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16).to(device)

# 모델의 특정 파라미터 업데이트 진행 X
for param in model.vision_tower.parameters():
    param.requires_grad = False

for param in model.multi_modal_projector.parameters():
    param.requires_grad = False


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

QLora 적용

In [ ]:
from transformers import BitsAndBytesConfig
from peft import get_peft_model, LoraConfig

# QLora 적용 (학습을 진행하는 인프라를 고려하여 QLora 적용)

# 4 bit 양자화 적용
bnb_config = BitsAndBytesConfig(        # 4-bit quantization
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

# Lora 설정
lora_config = LoraConfig(
    r=8, # r 차원
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], # Lora 적용대상
    task_type="CAUSAL_LM",
)
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
#trainable params: 11,298,816 || all params: 2,934,634,224 || trainable%: 0.38501616002417344


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 11,298,816 || all params: 2,934,765,296 || trainable%: 0.3850


 TrainingArguments 클래스를 사용하여 모델 학습에 대한 다양한 하이퍼파라미터를 설정

In [ ]:
from transformers import TrainingArguments
args=TrainingArguments(
            num_train_epochs=2,
            remove_unused_columns=False,
            per_device_train_batch_size=4,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            learning_rate=2e-5,
            weight_decay=1e-6,
            adam_beta2=0.999,
            logging_steps=100,
            optim="adamw_hf",
            save_strategy="steps",
            push_to_hub=True,
            save_steps=1000,
            save_total_limit=1,
            output_dir="paligemma_deepfake_2024_image_info",
            bf16=True,
            dataloader_pin_memory=False
        )


학습 진행

In [ ]:
from transformers import Trainer

trainer = Trainer(
        model=model,
        train_dataset=train_ds ,
        data_collator=collate_fn,
        args=args
        )


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Step,Training Loss
100,1.913400
200,0.304400
300,0.281400
400,0.265700
500,0.235400
600,0.226500
700,0.198400
800,0.186000
900,0.157100
1000,0.169200


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


Step,Training Loss
100,1.913400
200,0.304400
300,0.281400
400,0.265700
500,0.235400
600,0.226500
700,0.198400
800,0.186000
900,0.157100
1000,0.169200


TrainOutput(global_step=1362, training_loss=0.3277701946432195, metrics={'train_runtime': 23436.3121, 'train_samples_per_second': 0.931, 'train_steps_per_second': 0.058, 'total_flos': 8.43576117492914e+16, 'train_loss': 0.3277701946432195, 'epoch': 1.9977997799779978})

인퍼런스에 사용하기 위하여 huggingface에 모델 업로드

In [ ]:
trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/Donguri-b/paligemma_deepfake_2024_image_info/commit/76e095d03a4f1e8f22573e253d247d1b456c5858', commit_message='End of training', commit_description='', oid='76e095d03a4f1e8f22573e253d247d1b456c5858', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# model_id = "paligemma_deepfake_2024"
# model = PaliGemmaForConditionalGeneration.from_pretrained(model_id)
# processor = AutoProcessor.from_pretrained("google/paligemma-3b-pt-224")